In [1]:
# interactive figures, requires ipypml!
%matplotlib widget
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd
from gen_data import get_data, fit

In [3]:
d = get_data(25)
d

<xarray.DataArray (control: 25, time: 4112)>
array([[ 1.98606719,  1.99543536,  1.97105165, ...,  0.06721466,
         0.01622443, -0.01527595],
       [ 2.00662585,  1.99671981,  1.98191412, ..., -0.01417291,
         0.06389219, -0.04610717],
       [ 1.94324607,  2.08638654,  2.01925267, ...,  0.04242672,
        -0.01418673,  0.03341085],
       ...,
       [ 2.04979997,  2.03216639,  2.03112491, ..., -0.08414144,
         0.02488465, -0.03114146],
       [ 1.96096833,  2.00519296,  1.95848007, ..., -0.0852745 ,
        -0.00315458,  0.06245407],
       [ 2.00292556,  1.88319855,  1.92630386, ..., -0.03551658,
         0.03480358, -0.03723755]])
Coordinates:
  * control  (control) float64 15.0 15.62 16.25 16.88 ... 28.12 28.75 29.38 30.0
  * time     (time) float64 0.0 0.007642 0.01528 0.02293 ... 31.4 31.41 31.42

In [5]:
fig, ax = plt.subplots()
ax.plot(d.T)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
m = d[6]
m

<xarray.DataArray (time: 4112)>
array([ 1.97853570e+00,  2.04263441e+00,  1.95348930e+00, ...,
        1.16210873e-03, -5.68195846e-02, -6.30754001e-02])
Coordinates:
    control  float64 18.75
  * time     (time) float64 0.0 0.007642 0.01528 0.02293 ... 31.4 31.41 31.42

In [9]:
fig, ax = plt.subplots()
# ax.plot(m)
# ax.plot(m.time, m)
m.plot(ax=ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
fig, ax = plt.subplots()
control = float(m.control)
t = m.time
z = m.values
(ln,) = ax.plot(t, z, label=f"C: {control:.1f}")
ax.legend()
ax.set_xlabel("time (ms)")
ax.set_ylabel("displacement (mm)")


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'displacement (mm)')

In [11]:
def plot_one(ax, m):
    # pull what we want out of the xarray
    control = float(m.control)
    t = m.time
    z = m.values

    (ln,) = ax.plot(t, z, label=f"C: {control:.1f}")

    return {"raw": ln}


fig, ax = plt.subplots()
plot_one(ax, d[0])
plot_one(ax, d[5])
plot_one(ax, d[-1])
ax.legend()
ax.set_xlabel("time (ms)")
ax.set_ylabel("displacement (mm)")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'displacement (mm)')

In [12]:
def plot_one(ax, m, offset=0):
    # pull what we want out of the xarray
    control = float(m.coords["control"])
    t = m.time
    z = m.values

    (ln,) = ax.plot(t, z + offset, label=f"C: {control:.1f}")

    return {"raw": ln}


fig, ax = plt.subplots()
plot_one(ax, d[0])
plot_one(ax, d[10], offset=4)
plot_one(ax, d[20], offset=8)
ax.legend()
ax.set_xlabel("time (ms)")
ax.set_ylabel("displacement (mm)")


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'displacement (mm)')

In [13]:
from gen_data import fit

In [14]:
fit_vals = fit(d[6])
print(repr(fit_vals))
print(fit_vals)
print(fit_vals._repr_latex_())
fit_vals

Params(A=2.002055853953679, zeta=0.043700066728527456, omega=3.561814944214661, phi=1.576163437079454)
(A=2.00, ζ=0.04, ω0=3.56, φ=1.58)
$(A=2.00, \zeta=0.04, \omega_0=3.56,\varphi=1.58)$


Params(A=2.002055853953679, zeta=0.043700066728527456, omega=3.561814944214661, phi=1.576163437079454)

In [15]:
fig, ax = plt.subplots()
ax.plot(m.time, fit_vals.sample(m.time), label=fit_vals._repr_latex_(), color='k')
plt.gca().set_title(
     r"$A e^{-\zeta\omega_0t} \sin\left(\sqrt{1 - \zeta^2}\omega_0t + \varphi\right)$",
     usetex=True,
)

plt.legend()
ax.set_xlabel("time (ms)")
ax.set_ylabel("displacement (mm)")


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'displacement (mm)')

In [16]:
def plot_one(ax, m, popt, offset=0):
    # pull what we want out of the xarray
    control = float(m.coords["control"])
    t = m.coords["time"]
    z = m.values

    (ln,) = ax.plot(t, z + offset, label=f"C: {control:.1f}")
    (fit,) = ax.plot(t, popt.sample(t) + offset, color="k")
    ann = ax.annotate(
        f"$\\zeta={popt.zeta:.2g}$, $\\omega_0={popt.omega:.2f}$",
        # units are (axes-fraction, data)
        xy=(0.95, offset + 0.5),
        xycoords=ax.get_yaxis_transform(),
        # set the text alignment
        ha="right",
        va="bottom",
    )
    return {"raw": ln, "fit": fit, "annotation": ann}


In [17]:
fig, ax = plt.subplots()
plot_one(ax, d[0], fit(d[0]))
plot_one(ax, d[10], fit(d[10]), offset=4)
plot_one(ax, d[20], fit(d[20]), offset=8)
plot_one(ax, d[24], fit(d[24]), offset=12)
ax.legend(ncol=3, loc="upper center")
ax.set_xlabel("time (ms)")
ax.set_ylabel("displacement (mm)")
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
def plot_several(ax, d, fits):
    out = []

    for j, (m, popt) in enumerate(zip(d, fits)):
        arts = plot_one(ax, m, popt, offset=3.75 * j)
        out.append(arts)

    ax.set_xlabel("time (ms)")
    ax.set_ylabel("displacement (mm)")

    return out


In [20]:
fig, ax = plt.subplots()
indx = [0, 5, 10]
plot_several(ax, d[indx], [fit(d[i]) for i in indx])
ax.legend(ncol=3, loc="upper center")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
fits = [fit(m) for m in d]                               # fit all of the curves
fits_df = pd.DataFrame(fits, index=d.coords["control"])  # put the fit values is a DataFrame

In [22]:
fig, ax = plt.subplots()
fits_df.plot(y=['zeta', 'omega'], ax=ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='control'>

In [23]:
def plot_zeta(ax, fits_df):
    ax.set_ylabel(r"$\zeta$")
    ax.set_xlabel("control (arb)")
    ax.set_ylim(0, 0.08)
    return ax.plot(
        fits_df["zeta"], marker="o", color="k", label="\N{greek small letter zeta}", linestyle=''
    )


def plot_omega(ax, fits_df):
    ax.set_ylabel(r"$\omega_0/2\pi$ (kHz)")
    ax.set_xlabel("control (arb)")
    ax.set_ylim(0, 1.25)
    return ax.plot(
        fits_df["omega"] / (2 * np.pi),
        marker="o",
        color="k",
        label="\N{greek small letter omega}",
    )


In [24]:
fig, (ax1, ax2) = plt.subplots(1, 2, constrained_layout=True)
plot_zeta(ax1, fits_df)
plot_omega(ax2, fits_df)
ax1.legend()
ax2.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [25]:
def plot_one(ax, m, popt, offset=0):
    # pull what we want out of the xarray
    control = float(m.coords["control"])
    t = m.coords["time"]
    z = m.values

    (ln,) = ax.plot(t, z + offset, label=f"C: {control:.1f}")
    (fit,) = ax.plot(t, popt.sample(t) + offset, color="k")
    ann = ax.annotate(
        f"$C={control:.1f}$\n$\\zeta={popt.zeta:.2g}$ $\\omega_0={popt.omega:.2f}$",
        # units are (axes-fraction, data)
        xy=(0.95, offset + 0.5),
        xycoords=ax.get_yaxis_transform(),
        # set the text alignment
        ha="right",
        va="bottom",
    )
    return {"raw": ln, "fit": fit, "annotation": ann}

def plot_several(ax, d, fits):
    out = []

    for j, (m, popt) in enumerate(zip(d, fits)):
        arts = plot_one(ax, m, popt, offset=3.75 * j)
        out.append(arts)

    ax.set_xlabel("time (ms)")
    ax.set_ylabel("displacement (mm)")

    return out

In [26]:
fig, ax = plt.subplots()
indx = [0, 10, 24]
plot_several(ax, d[indx], [fits[i] for i in indx])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[{'raw': <matplotlib.lines.Line2D at 0x7f15e06525b0>,
  'fit': <matplotlib.lines.Line2D at 0x7f15e0652880>,
  'annotation': Text(0.95, 0.5, '$C=15.0$\n$\\zeta=0.052$ $\\omega_0=3.00$')},
 {'raw': <matplotlib.lines.Line2D at 0x7f15e0652b50>,
  'fit': <matplotlib.lines.Line2D at 0x7f15e06527f0>,
  'annotation': Text(0.95, 4.25, '$C=21.2$\n$\\zeta=0.049$ $\\omega_0=4.01$')},
 {'raw': <matplotlib.lines.Line2D at 0x7f15e06524c0>,
  'fit': <matplotlib.lines.Line2D at 0x7f15e06596a0>,
  'annotation': Text(0.95, 8.0, '$C=30.0$\n$\\zeta=0.058$ $\\omega_0=6.00$')}]

In [27]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, constrained_layout=True)
indx = [0, 10, 24]
plot_several(ax1, d[indx], [fits[i] for i in indx])
plot_zeta(ax2, fits_df)
plot_omega(ax3, fits_df)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [28]:
fig, ax_dict = plt.subplot_mosaic("""
AB
AC
""", constrained_layout=True)
indx = [0, 10, 24]
plot_several(ax_dict['A'], d[indx], [fits[i] for i in indx])
plot_zeta(ax_dict['B'], fits_df)
plot_omega(ax_dict['C'], fits_df)
fig.align_ylabels(list(ax_dict.values()))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [29]:
def subplot_label(ax, text):
    return ax.annotate(
        text,
        # units are (axes-fraction, axes-fraction)
        # # this is bottom right
        # xy=(1, 0),
        # this is the top left
        xy=(0, 1),
        xycoords="axes fraction",
        # units are absolute offset in points from xy
        xytext=(-5, 5),
        textcoords=("offset points"),
        # set the text alignment
        ha="right",
        va="bottom",
        fontweight="bold",
        fontsize="larger",
    )


In [30]:
fig, ax_dict = plt.subplot_mosaic("""
AB
AC
""", constrained_layout=True)
indx = [0, 10, 24]
plot_several(ax_dict['A'], d[indx], [fits[i] for i in indx])
plot_zeta(ax_dict['B'], fits_df)
plot_omega(ax_dict['C'], fits_df)
fig.align_ylabels(list(ax_dict.values()))
subplot_labels = {
    k: subplot_label(v, f"({k.lower()})") for k, v in ax_dict.items()
}

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [31]:
def paper_figure_2(fig, layout, d, fits, *, plot_every=5):
    ax_dict = fig.subplot_mosaic(layout)

    fits_df = pd.DataFrame(fits, index=d.coords["control"])

    index = list(range(0, len(d), plot_every))

    artists = {
        "vibrations": plot_several(ax_dict["A"], d[index], [fits[i] for i in index]),
        "zeta": plot_zeta(ax_dict["B"], fits_df),
        "omega": plot_omega(ax_dict["C"], fits_df),
    }

    fig.align_ylabels(list(ax_dict.values()))
    subplot_labels = {
        k: subplot_label(v, f"({k.lower()})") for k, v in ax_dict.items()
    }

    return (fig, ax_dict, artists, subplot_labels)

In [32]:
single_col_width = 8.6 / 2.54  # single column APS figure
double_col_width = 17.8 / 2.54  # double column APS figure


In [33]:
fig, axs, arts, labels = paper_figure_2(
    plt.figure(
        constrained_layout=True, figsize=(double_col_width, double_col_width * 0.5)
    ),
    "AB\nAC",
    d,
    fits,
    plot_every=5,
)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [34]:
paper_figure_2(
    plt.figure(
        constrained_layout=True, figsize=(single_col_width, single_col_width * 2.5)
    ),
    "A\nB\nC",
    d,
    fits,
    plot_every=10,
)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(<Figure size 338.583x846.457 with 3 Axes>,
 {'A': <AxesSubplot:label='A', xlabel='time (ms)', ylabel='displacement (mm)'>,
  'B': <AxesSubplot:label='B', xlabel='control (arb)', ylabel='$\\zeta$'>,
  'C': <AxesSubplot:label='C', xlabel='control (arb)', ylabel='$\\omega_0/2\\pi$ (kHz)'>},
 {'vibrations': [{'raw': <matplotlib.lines.Line2D at 0x7f159c182eb0>,
    'fit': <matplotlib.lines.Line2D at 0x7f159c1064c0>,
    'annotation': Text(0.95, 0.5, '$C=15.0$\n$\\zeta=0.052$ $\\omega_0=3.00$')},
   {'raw': <matplotlib.lines.Line2D at 0x7f159c1069a0>,
    'fit': <matplotlib.lines.Line2D at 0x7f159c106df0>,
    'annotation': Text(0.95, 4.25, '$C=21.2$\n$\\zeta=0.049$ $\\omega_0=4.01$')},
   {'raw': <matplotlib.lines.Line2D at 0x7f159c10c1f0>,
    'fit': <matplotlib.lines.Line2D at 0x7f159c10cb20>,
    'annotation': Text(0.95, 8.0, '$C=27.5$\n$\\zeta=0.056$ $\\omega_0=5.36$')}],
  'zeta': [<matplotlib.lines.Line2D at 0x7f159c10c460>],
  'omega': [<matplotlib.lines.Line2D at 0x7f159c10cf40>]},